# CHAPTER 06 머신러닝, 딥러닝 기초 이론

## 8. 합성곱 신경망

In [1]:
import numpy as np
import tensorflow as tf
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

C:\Users\Daeung\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### MNIST 데이터

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("/tmp/data/")

X_train = mnist.train.images
X_val = mnist.validation.images
X_test = mnist.test.images
y_train = mnist.train.labels.astype("int")
y_val = mnist.validation.labels.astype("int")
y_test = mnist.test.labels.astype("int")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


### CNN 구현

In [4]:
reset_graph()

height = 28
width = 28
channels = 1
n_inputs = height * width * channels
n_outputs = 10

X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
y = tf.placeholder(tf.int32, shape=[None], name="y")


conv1 = tf.layers.conv2d(X_reshaped, filters=32, kernel_size=3,
                         strides=1, padding="SAME",
                         activation=tf.nn.relu, name="conv1")

conv2 = tf.layers.conv2d(conv1, filters=64, kernel_size=3,
                         strides=2, padding="SAME",
                         activation=tf.nn.relu, name="conv2")

pool3 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")

pool3_shape = pool3.get_shape().as_list()
pool3_flat = tf.reshape(pool3, shape=[-1, np.prod(pool3_shape[1:])])

fc1 = tf.layers.dense(pool3_flat, 64, activation=tf.nn.relu, name="fc1")
logits = tf.layers.dense(fc1, n_outputs, name="output")


xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
loss = tf.reduce_mean(xentropy)
optimizer = tf.train.AdamOptimizer()
training_op = optimizer.minimize(loss)

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
saver = tf.train.Saver()

In [6]:
init = tf.global_variables_initializer()

n_epochs = 10
batch_size = 32

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run([training_op], 
                     feed_dict={X: X_batch, y: y_batch})
            
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: X_val, y: y_val})
        print(epoch, "Train accuracy:", acc_train, "Val accuracy:", acc_val)

    save_path = saver.save(sess, "./models/model_final.ckpt")

0 Train accuracy: 0.96875 Val accuracy: 0.9836
1 Train accuracy: 0.96875 Val accuracy: 0.9846
2 Train accuracy: 1.0 Val accuracy: 0.9846
3 Train accuracy: 1.0 Val accuracy: 0.9878
4 Train accuracy: 1.0 Val accuracy: 0.9864
5 Train accuracy: 1.0 Val accuracy: 0.9878
6 Train accuracy: 1.0 Val accuracy: 0.987
7 Train accuracy: 1.0 Val accuracy: 0.9902
8 Train accuracy: 1.0 Val accuracy: 0.9896
9 Train accuracy: 1.0 Val accuracy: 0.9894
